<a href="https://colab.research.google.com/github/harshitEbPandey/transferLearningNER/blob/mlp_models/MLP_hindi_all_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
!pip install fasttext

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import optimizers
from keras import regularizers
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import fasttext
from tensorflow import random

### Tokenizing and saving txt (X_train_hindi, Y_train_hindi)

In [ ]:
from datasets import load_dataset
hi_ner = load_dataset('ai4bharat/naamapadam', 'hi')

In [ ]:
# analysing the hindi dataset
hindi_ds = hi_ner.map()

# shapes of each
print(f"Train: {hindi_ds['train'].shape}")
print(f"Test: {hindi_ds['test'].shape}")
print(f"Validation: {hindi_ds['validation'].shape}")

# loading each
train_hindi_ds = hindi_ds['train'].to_pandas()
test_hindi_ds = hindi_ds['test'].to_pandas()
vali_hindi_ds = hindi_ds['validation'].to_pandas()

In [ ]:
# combine each of the words in the individual sentences and create a common df
# text output is of the form in X_train.txt: `tokens`
# text output is of the form in Y_train.txt: `ner_tags`
import pandas as pd
with open('/content/drive/MyDrive/Spring 2023/CSE 572 data mining/Project/Final/X_train_hindi_ds.txt', 'w') as f1, open('/content/drive/MyDrive/Spring 2023/CSE 572 data mining/Project/Final/Y_train_hindi_ds.txt', 'w') as f2:
  for index, row in train_hindi_ds.iterrows():
      f1.write('\n'.join(row['tokens']) + '\n')
      f2.write('\n'.join(map(str, row['ner_tags'])) + '\n')

### Loading the tokens and tags from txt

In [ ]:
file_loc = '/content/drive/MyDrive/Spring 2023/CSE 572 data mining/Project/Final'

In [ ]:
# all tokens and tags are presaved...
# taking them input and making unique dictionary
# Open the input file and read the words
with open(file_loc + '/X_train_hindi_ds.txt', 'r') as f:
    words = f.read().splitlines()

In [ ]:
with open(file_loc + '/Y_train_hindi_ds.txt', 'r') as f:
    tags = f.read().splitlines()

In [ ]:
len(tags) == len(words)

True

In [ ]:
word_tag_unique = {}
for idx, word in enumerate(words):
  if word not in word_tag_unique:
    word_tag_unique[word] = [tags[idx]]
  elif word in word_tag_unique and tags[idx] not in word_tag_unique[word]:
    word_tag_unique[word].append(tags[idx])

In [ ]:
with open(file_loc + '/train_hindi_unique_tokens.txt', 'w') as f:
  for word in word_tag_unique.keys():
    f.write(word + '\n')

### Training fasttext on unique words dimensions = 30

In [ ]:
# run fasttext model
model = fasttext.train_unsupervised(file_loc + '/train_hindi_unique_tokens.txt', model='skipgram', lr=0.05, dim=30, ws=5, epoch=5)

In [ ]:
# save the model
model.save_model(file_loc + '/fasttext_model_train_hindi_unique_tokens.bin')

In [ ]:
# load the model
model = fasttext.load_model(file_loc + '/fasttext_model_train_hindi_unique_tokens.bin')

In [ ]:
# every word should have an embedding now
# can get using model[word]

### Loading the training set

In [ ]:
# constructing x_train_df

word_dict = {i: word for i, word in enumerate(words)}
X_train_df = pd.DataFrame.from_dict(word_dict, orient='index', columns=['tokens'])

In [ ]:
# constructing y_train_df

tag_dict = {i: tag for i, tag in enumerate(tags)}
Y_train_df = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

### Saving word vectors in chunks

In [ ]:
def process_chunk(start_idx, end_idx, df):
    # Get the words for the chunk
    words = df.loc[start_idx:end_idx, 'tokens'].tolist()

   # Get the vectors for the words using the fastText model
    vectors = [model[word] for word in words]

    # list of index
    idx = range(start_idx, end_idx + 1)

    # Create a new dataframe with the vectors
    vectors_df = pd.DataFrame({'tokens': words, 'vectors': vectors}, index = idx)

    # # Save the chunk to a CSV file
    vectors_df.to_csv(file_loc + f'/vectors/hindi_lite_vectors_{start_idx}_{end_idx}.csv', index=False)

In [ ]:
X_train_df.shape

(22029408, 1)

In [ ]:
# Loop over the dataframe in chunks
chunk_size = 1000000
for i in range(0, len(X_train_df), chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, len(X_train_df)-1)
    process_chunk(start_idx, end_idx, X_train_df)
    # Print progress
    print(f'Saved vectors for words {start_idx} to {end_idx} to CSV file.')

Saved vectors for words 0 to 1000000 to CSV file.
Saved vectors for words 1000000 to 2000000 to CSV file.
Saved vectors for words 2000000 to 3000000 to CSV file.
Saved vectors for words 3000000 to 4000000 to CSV file.
Saved vectors for words 4000000 to 5000000 to CSV file.
Saved vectors for words 5000000 to 6000000 to CSV file.
Saved vectors for words 6000000 to 7000000 to CSV file.
Saved vectors for words 7000000 to 8000000 to CSV file.
Saved vectors for words 8000000 to 9000000 to CSV file.
Saved vectors for words 9000000 to 10000000 to CSV file.
Saved vectors for words 10000000 to 11000000 to CSV file.
Saved vectors for words 11000000 to 12000000 to CSV file.
Saved vectors for words 12000000 to 13000000 to CSV file.
Saved vectors for words 13000000 to 14000000 to CSV file.
Saved vectors for words 14000000 to 15000000 to CSV file.
Saved vectors for words 15000000 to 16000000 to CSV file.
Saved vectors for words 16000000 to 17000000 to CSV file.
Saved vectors for words 17000000 to 180

In [ ]:
# encode the labels
label_encoder = LabelEncoder()
label_encoder.fit(Y_train_df.tags.values)
Y_train_hindi_enc = label_encoder.transform(Y_train_df.tags.values)

In [ ]:
# get each class' weights
unique, counts = np.unique(Y_train_hindi_enc, return_counts=True)
class_weights = dict(zip(unique, np.round(sum(counts) / counts)))

# more the number of records - lesser the weights are (for balancing dataset)
print(class_weights)

{0: 1.0, 1: 29.0, 2: 31.0, 3: 32.0, 4: 27.0, 5: 30.0, 6: 106.0}


In [ ]:
Y_train_hindi_enc.shape

(22029408,)

### Training MLP CLassifier

In [ ]:
np.unique(Y_train_hindi_enc)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
num_features = 30
print(num_features)
num_classes = 7

30


In [ ]:
# Set the random seed
np.random.seed(42)
random.set_seed(42)

mlp = Sequential()
mlp.add(Dense(units=100, activation='relu', input_dim=num_features))
mlp.add(Dense(units=num_classes, activation='softmax'))
mlp.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Load test data

In [ ]:
# Load the pre-trained fastText model
model = fasttext.load_model(file_loc + '/fasttext_model_train_hindi_unique_tokens.bin')

# Open the input file and read the words
with open(file_loc + '/X_test_hindi_ds.txt', 'r') as f:
    words = [line.strip() for line in f if line.strip()]

# Create a dictionary with the words as values and integer indices as keys
word_dict = {i: word for i, word in enumerate(words)}

# Create a dataframe from the dictionary
df_test = pd.DataFrame.from_dict(word_dict, orient='index', columns=['tokens'])

# Loop over the dataframe in chunks
start_idx = 0
end_idx = len(df_test) - 1
process_chunk(start_idx, end_idx, df_test)
# Print progress
print(f'Saved test vectors for words {start_idx} to {end_idx} to CSV file.')

# get the labels in another dataframe
with open(file_loc +'/Y_test_hindi_ds.txt', 'r') as f:
    test_tags = [line.strip() for line in f if line.strip()]

tag_dict = {i: tag for i, tag in enumerate(test_tags)}

df_test_tags = pd.DataFrame.from_dict(tag_dict, orient='index', columns=['tags'])

X_test = pd.read_csv(file_loc + f'/vectors/hindi_lite_vectors_{start_idx}_{end_idx}.csv')
X_test.head()

Saved test vectors for words 0 to 8404 to CSV file.


,tokens,vectors
0,गुजरात,[ 3.5765130e-04 -9.3661115e-04 1.6989829e-03 ...
1,हाईकोर्ट,[ 2.2866970e-03 9.4011496e-04 1.8860205e-04 ...
2,ने,[ 0.00794061 -0.00770023 0.00818515 0.005586...
3,मंगलवार,[-1.36724091e-04 1.14136667e-03 -7.04363483e-...
4,को,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [ ]:
# preparing data for prediction

target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5', 'Class 6']

y_test_hindi_ds = label_encoder.transform(df_test_tags.tags.values)

X_test = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_test.vectors])

Y_test = y_test_hindi_ds

y_true_labels = Y_test

In [ ]:
# training model pipeline (in chunks)

# Load data in chunks, and train
# training sets
# Loop over the dataframe in chunks
chunk_size = 1000000
for i in range(0, len(X_train_df), chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, len(X_train_df)-1)
    X_train = pd.read_csv(file_loc + f'/vectors/hindi_lite_vectors_{start_idx}_{end_idx}.csv')
    X_train = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_train.vectors])
    Y_train = Y_train_hindi_enc[start_idx: end_idx + 1]

    # Print progress
    print(f'Loaded vectors for words {start_idx} to {end_idx} to CSV file.')
    
    # train mlp on this
    mlp.fit(X_train, Y_train, class_weight=class_weights, epochs = 50, batch_size = 200)

    # predict on test
    Y_test_pred = mlp.predict(X_test)

    # get pred labels
    y_pred_labels = np.argmax(Y_test_pred, axis=1)

    # print classification report
    print(classification_report(y_true_labels, y_pred_labels, target_names=target_names))

    # save model
    mlp.save(file_loc + f'/models/mlp_model_hindi_lite_50_epochs.h5')

    # Print progress
    print(f'Completed training with vectors for words {start_idx} to {end_idx}')

Loaded vectors for words 0 to 1000000 to CSV file.
Epoch 1/50
5001/5001 [==============================] - 14s 3ms/step - loss: 13.1573 - accuracy: 0.0859
Epoch 2/50
5001/5001 [==============================] - 16s 3ms/step - loss: 12.9901 - accuracy: 0.1022
Epoch 3/50
5001/5001 [==============================] - 14s 3ms/step - loss: 12.8891 - accuracy: 0.1005
Epoch 4/50
5001/5001 [==============================] - 14s 3ms/step - loss: 12.8120 - accuracy: 0.1018
Epoch 5/50
5001/5001 [==============================] - 13s 3ms/step - loss: 12.7386 - accuracy: 0.1055
Epoch 6/50
5001/5001 [==============================] - 15s 3ms/step - loss: 12.6595 - accuracy: 0.1113
Epoch 7/50
5001/5001 [==============================] - 16s 3ms/step - loss: 12.5847 - accuracy: 0.1172
Epoch 8/50
5001/5001 [==============================] - 18s 4ms/step - loss: 12.5017 - accuracy: 0.1450
Epoch 9/50
5001/5001 [==============================] - 16s 3ms/step - loss: 12.4226 - accuracy: 0.2216
Epoch 10/50
5

### Baseline report 1 :P Did not save the model

```
     Class 0       0.92      0.70      0.80      6996
     Class 1       0.18      0.42      0.25       263
     Class 2       0.21      0.31      0.25       239
     Class 3       0.16      0.40      0.23       257
     Class 4       0.16      0.41      0.23       253
     Class 5       0.27      0.43      0.33       302
     Class 6       0.11      0.32      0.16        95
    accuracy                           0.65      8405
   macro avg       0.28      0.43      0.32      8405
weighted avg       0.80      0.65      0.70      8405
```
Completed training with vectors for words 7000000 to 8000000 (200 epoochs each
)

In [ ]:
# train for another 100 epochs, last 29000 tokens 
chunk_size = 1000000
for i in range(22000000, len(X_train_df), chunk_size):
    start_idx = i
    end_idx = min(i + chunk_size, len(X_train_df)-1)
    X_train = pd.read_csv(file_loc + f'/vectors/hindi_lite_vectors_{start_idx}_{end_idx}.csv')
    X_train = np.array([np.fromstring(s[1:-1].replace('\n', ' '), sep=' ') for s in X_train.vectors])
    Y_train = Y_train_hindi_enc[start_idx: end_idx + 1]

    # Print progress
    print(f'Loaded vectors for words {start_idx} to {end_idx} to CSV file.')
    
    # train mlp on this
    mlp.fit(X_train, Y_train, class_weight=class_weights, epochs = 150, batch_size = 200)

    # predict on test
    Y_test_pred = mlp.predict(X_test)

    # get pred labels
    y_pred_labels = np.argmax(Y_test_pred, axis=1)

    # print classification report
    print(classification_report(y_true_labels, y_pred_labels, target_names=target_names))

    # save model
    mlp.save(file_loc + f'/models/mlp_model_hindi_lite_50_epochs_retrain.h5')

    # Print progress
    print(f'Completed training with vectors for words {start_idx} to {end_idx}')

Loaded vectors for words 22000000 to 22029407 to CSV file.
Epoch 1/150
148/148 [==============================] - 0s 2ms/step - loss: 10.3283 - accuracy: 0.6000
Epoch 2/150
148/148 [==============================] - 0s 3ms/step - loss: 10.3819 - accuracy: 0.4622
Epoch 3/150
148/148 [==============================] - 0s 2ms/step - loss: 10.3035 - accuracy: 0.5624
Epoch 4/150
148/148 [==============================] - 0s 2ms/step - loss: 10.3005 - accuracy: 0.5257
Epoch 5/150
148/148 [==============================] - 0s 2ms/step - loss: 10.3005 - accuracy: 0.5166
Epoch 6/150
148/148 [==============================] - 0s 2ms/step - loss: 10.2932 - accuracy: 0.6095
Epoch 7/150
148/148 [==============================] - 0s 2ms/step - loss: 10.3028 - accuracy: 0.5178
Epoch 8/150
148/148 [==============================] - 0s 2ms/step - loss: 10.2824 - accuracy: 0.5586
Epoch 9/150
148/148 [==============================] - 0s 2ms/step - loss: 10.2836 - accuracy: 0.6299
Epoch 10/150
148/148 [=

### Final good baseline -- results on 50 epochs -- and 150 epoch final 29000 retrained:
```
              precision    recall  f1-score   support

     Class 0       0.92      0.71      0.80      6996
     Class 1       0.18      0.38      0.25       263
     Class 2       0.19      0.31      0.23       239
     Class 3       0.13      0.47      0.20       257
     Class 4       0.18      0.36      0.24       253
     Class 5       0.26      0.41      0.32       302
     Class 6       0.15      0.29      0.20        95

    accuracy                           0.65      8405
   macro avg       0.29      0.42      0.32      8405
weighted avg       0.80      0.65      0.71      8405
```